In [51]:
# Librerias necesarias
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [52]:
# Ruta de los datos:
ruta_games= 'Datasets originales/steam_games.json.gz'
ruta_user_reviews= 'Datasets originales/user_reviews.json.gz'
ruta_user_items= 'Datasets originales/users_items.json.gz'

#### Games

In [53]:
## Games:
games_orig = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo_comprimido:
    for linea in archivo_comprimido:
        diccionario = json.loads(linea)
        games_orig.append(diccionario)

# Transformamos a DataFrame:
games_orig = pd.DataFrame(games_orig)

In [98]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
games = games_orig.copy()

In [ ]:
games.tail()

Las transformaciones que realizaremos en este caso serán: 

1. Eliminar registros completamente vacíos. 
2. Eliminar columnas que no utilizaremos: ``publisher``, ``url``, ``reviews_url``, ``price``, ``early_access``, ``developer``. Eliminar la columna ``tags`` previo a rellenar los datos faltantes de genres con sus valores. ``app_name`` y ``title`` dicen lo mismo, dejaremos la columna que tenga menor cantidad de datos faltantes.
3. Desanidar registros que poseen valores con tipo de dato LISTA.
4. Eliminar registros duplicados.
5. Corregir el tipo de dato de cada columna. Los registros que poseen el campo ``release_date`` vacío o con un dato que no corresponde, se rellenaran con '1900-01-01' para poder mantener el registro sin perjudicar el análisis. También creamos ``release_year``.
6. Eliminar registros que poseen datos vacíos en columnas relevantes.

In [99]:
# 1. Eliminar registros completamente vacíos
print(f'tamaño inicial del dataframe: {len(games)} registros')
games.dropna(how='all', inplace= True, ignore_index=True)
print(f'tamaño final del dataframe: {len(games)} registros ')

tamaño inicial del dataframe: 120445 registros
tamaño final del dataframe: 32135 registros 


In [100]:
# 2. Eliminar columnas que no utilizaremos: ``publisher``, ``url``, ``reviews_url``, ``price``, ``early_access``, ``developer``.
# Eliminar la columna ``tags`` previo a rellenar los datos faltantes de ``genres`` con sus valores. 
# ``app_name`` y ``title`` dicen lo mismo, dejaremos "app_name" que tiene menos datos faltantes y le cambiaremos el nombre a "title"

print(f"El tamaño inicial de games era {games.shape}")

games['genres'] = games['genres'].combine_first(games['tags']) # a cada valor faltante de genres le hara corresponder lo que figure en tags
games['title']= games['app_name']
games.drop(columns=['publisher', 'url', 'reviews_url', 'price', 'early_access', 'developer', 'tags', 'app_name'], inplace=True)

print("Columnas sin utilidad eliminadas")
print(f"Ahora, el tamaño final es: {games.shape}")


El tamaño inicial de games era (32135, 13)
Columnas sin utilidad eliminadas
Ahora, el tamaño final es: (32135, 5)


In [101]:
games.sample()

,genres,title,release_date,specs,id
17589,"[Casual, Simulation]",Rocksmith® 2014 Edition – Remastered – Queen -...,2017-05-23,"[Single-player, Shared/Split Screen, Downloada...",590224


In [102]:
# 3. Desanidar registros que poseen valores con tipo de dato LISTA: genres y specs.
games = games.explode(column=['genres'], ignore_index=True)
games = games.explode(column=['specs'], ignore_index=True)
games.shape

(408844, 5)

In [103]:
# 4. Eliminar registros duplicados.
games.drop_duplicates(inplace=True)
games.shape

(408840, 5)

In [104]:
# 5. Corregir el tipo de dato de cada columna.
## genres OK
## specs OK
## title OK
## release_date INCONSISTENTE
## id VALORES NAN
fecha_relleno = pd.to_datetime('1900-01-01')
games['release_date'] = pd.to_datetime(games['release_date'], format='%Y-%m-%d', errors='coerce').fillna(fecha_relleno).dt.year
games.rename(columns={'release_date':'release_year'}, inplace=True)
# Si no tiene ID no sirve, se dropea.
games.dropna(subset='id', inplace=True)
games['id']=games['id'].astype(int)
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 408829 entries, 0 to 408843
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   genres        408631 non-null  object
 1   title         408825 non-null  object
 2   release_year  408829 non-null  int32 
 3   specs         407184 non-null  object
 4   id            408829 non-null  int32 
dtypes: int32(2), object(3)
memory usage: 15.6+ MB


In [107]:
# 6. Eliminar registros que poseen datos vacíos en columnas relevantes
games[games['genres'].isna()]

### ¿QUE HAGO CON GENRES?

,genres,title,release_year,specs,id
1911,NaN,Europa Universalis III: Heir to the Throne,2009,Single-player,25806
1912,NaN,Europa Universalis III: Heir to the Throne,2009,Multi-player,25806
1913,NaN,Europa Universalis III: Heir to the Throne,2009,Downloadable Content,25806
2424,NaN,Booster Trooper Demo,2010,Game demo,27930
3270,NaN,"Warhammer 40,000: Dawn of War II - Retribution...",2011,Downloadable Content,56436
...,...,...,...,...,...
403439,NaN,Total War: SHOGUN 2 - Sengoku Jidai Unit Pack,2011,Downloadable Content,34342
403931,NaN,"Worms Reloaded: The ""Pre-order Forts and Hats""...",2011,Downloadable Content,22630
403970,NaN,Total War: SHOGUN 2 - The Ikko Ikki Clan Pack,2011,Downloadable Content,34348
404322,NaN,"Killing Floor ""London's Finest"" Character Pack",2010,Downloadable Content,35419


## Para poder MERGEAR las tablas, los campos de las PK y FK deben ser el mismo tipo de dato. No puedo mergear un ``item_id`` string con un ``item_id`` integer

#### Reviews

In [68]:
## Reviews:
user_reviews = []
with gzip.open(ruta_user_reviews, 'rt', encoding="utf-8") as archivo_comprimido:
    for linea in archivo_comprimido:
        user_reviews.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_reviews_orig = pd.DataFrame(user_reviews)


In [69]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
user_reviews = user_reviews_orig.copy()

In [70]:
# Extraigo solo las columnas que voy a usar

reviews = user_reviews[['user_id','reviews']]
reviews.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [71]:
# Desanidamos reviews:

reviews_explode = reviews.explode('reviews', ignore_index=True) # separo en filas todos todos los json que estan listados en cada registro
reviews_desanidado = pd.json_normalize(reviews_explode['reviews']) # transformo a tabla cada uno de los json que tenemos en cada registro
reviews = pd.concat([reviews_explode,reviews_desanidado], axis=1).drop(columns=['reviews']) # concateno ambos dataframes y dropeo la columna reviews que está anidada

#### Items

In [ ]:
## Items:
user_items = []
with gzip.open(ruta_user_items, 'rt', encoding="utf-8") as archivo_comprimido:
    for linea in archivo_comprimido:
        user_items.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_items_orig = pd.DataFrame(user_items)

In [ ]:
# Copia de los datos para no estar constantemente cargandolos para resetearlos
user_items = user_items_orig.copy()

In [ ]:
# Extraigo solo las columnas que voy a usar:
items = user_items[['user_id','items']]
items.head()

In [ ]:
# Desanidamos items:

items_explode = items.explode('items', ignore_index=True) # separo en filas todos todos los json que estan listados en cada registro
items_desanidado = pd.json_normalize(items_explode['items']) # transformo a tabla cada uno de los json que tenemos en cada registro
items = pd.concat([items_explode,items_desanidado], axis=1).drop(columns=['items']) # concateno ambos dataframes y dropeo la columna items que está anidada

Ahora tenemos 3 tablas con las que trabajaremos: `games`, `reviews` e `items`

#### Transformaciones generales

In [ ]:
games['title'].info()
games['app_name'].info()
# Son columnas similares. En este caso la columna que voy a dejar es app_name porque posee menor cantidad de datos vacios: 2.

In [ ]:
# Prueba para rellenar valores faltantes de genres
prueba = games[['genres','tags']].loc[88313:88320]
prueba['genres'] = prueba['genres'].combine_first(prueba['tags'])
prueba

In [ ]:
games['genres'] = games['genres'].combine_first(games['tags']) # a cada valor faltante de genres le hara corresponder lo que figure en tags
games.drop(columns=['publisher'])

In [ ]:
# Tratamiento de registros vacíos
print(f"El tamaño inicial del dataset games es: \n{games.shape}")
print(f"El tamaño inicial del dataset reviews es: \n{reviews.shape}")
print(f"El tamaño inicial del dataset items es: \n{items.shape}")

games.dropna(how='all', inplace=True)
reviews.dropna(how='all', inplace=True)
items.dropna(how='all', inplace=True)

print(f"Removiendo los registros completamente vacíos nos quedaran con un tamaño de \n games: {games.shape}")
print(f"Removiendo los registros completamente vacíos nos quedaran con un tamaño de \n reviews: {reviews.shape}")
print(f"Removiendo los registros completamente vacíos nos quedaran con un tamaño de \n items: {items.shape}")

In [ ]:
# Tratamiento de registros duplicados
print(f"El tamaño del dataset games previo a realizar limpieza de registros duplicados es: {games.shape}")
print(f"El tamaño del dataset reviews previo a realizar limpieza de registros duplicados es: {reviews.shape}")
print(f"El tamaño del dataset items previo a realizar limpieza de registros duplicados es: {items.shape}")

games.drop_duplicates(subset=['publisher', 'app_name', 'title', 'url', 'release_date', 'reviews_url', 'price', 'early_access', 'id','developer'])
reviews.drop_duplicates()
items.drop_duplicates()

print(f"El tamaño del dataset games luego de realizar limpieza de registros duplicados es: {games.shape}")
print(f"El tamaño del dataset reviews luego de realizar limpieza de registros duplicados es: {reviews.shape}")
print(f"El tamaño del dataset items luego de realizar limpieza de registros duplicados es: {items.shape}")

#### Las tablas que finalmente utilizaremos son: ***games***, ***reviews*** e ***items***

#### 3. Carga o disponibilización de datos

#### 3.1 PlayTimeGenre:  Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [ ]:
games.tail()

In [ ]:
reviews.head()

In [ ]:
items.head()

#### 3.2 UsersForGenre: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [ ]:
items[['user_id','item_id','playtime_forever']].head()